In [1]:
path = 'C:\\Users\\under\\GitHub\\On-the-Curse-of-Sentence-Length\\out.txt'

In [3]:
import numpy as np

In [4]:
out = open(path, mode='rt', encoding='utf-8').read().strip().split('\n')

In [5]:
len(out)

120474

In [95]:
out[:8]
out_red = out[7:-4]

In [96]:
n = 4
out_pair = [out_red[i * n:(i + 1) * n] for i in range((len(out_red) + n - 1) // n )]

In [93]:
len(out_pair)

30117

In [9]:
for i in range(5):
    print(out_pair[i])

['S-11722\t', 'T-11722\t<<unk>> <<unk>> est nécessaire <<unk>> des modifications structurelles aux <<unk>> un délai limite de six mois sera <<unk>>', 'H-11722\t-2.477929473284364\tLe Conseil de l&apos; Europe', 'P-11722\t-5.0288 -4.9506 -2.3406 -1.1004 -0.1102 -1.3370']
['S-13133\t', 'T-13133\tAvec un carnet de commandes riche de 2 <<unk>> <<unk>> son plan de charge est assuré pour plusieurs <<unk>>', 'H-13133\t-2.477929473284364\tLe Conseil de l&apos; Europe', 'P-13133\t-5.0288 -4.9506 -2.3406 -1.1004 -0.1102 -1.3370']
['S-11834\t', 'T-11834\t- Madame la <<unk>> deux points méritent une attention particulière dans ce <<unk>>', 'H-11834\t-2.477929473284364\tLe Conseil de l&apos; Europe', 'P-11834\t-5.0288 -4.9506 -2.3406 -1.1004 -0.1102 -1.3370']
['S-27196\t', 'T-27196\tLa <<unk>> devrait élaborer une stratégie plus claire concernant les relations avec nos <<unk>>', 'H-27196\t-2.477929473284364\tLe Conseil de l&apos; Europe', 'P-27196\t-5.0288 -4.9506 -2.3406 -1.1004 -0.1102 -1.3370']


In [10]:
print(out_pair[0][0].split('\t'))
print(out_pair[0][1].split('\t'))
print(out_pair[0][2].split('\t'))

['S-11722', '']
['T-11722', '<<unk>> <<unk>> est nécessaire <<unk>> des modifications structurelles aux <<unk>> un délai limite de six mois sera <<unk>>']
['H-11722', '-2.477929473284364', 'Le Conseil de l&apos; Europe']


In [11]:
src = out_pair[0][0].split('\t')[1].split(' ')
ref = out_pair[0][1].split('\t')[1].split(' ')
hyp = out_pair[0][2].split('\t')[2].split(' ')
np.array([src, ref, hyp])

array([list(['']),
       list(['<<unk>>', '<<unk>>', 'est', 'nécessaire', '<<unk>>', 'des', 'modifications', 'structurelles', 'aux', '<<unk>>', 'un', 'délai', 'limite', 'de', 'six', 'mois', 'sera', '<<unk>>']),
       list(['Le', 'Conseil', 'de', 'l&apos;', 'Europe'])], dtype=object)

In [13]:
from torchtext.data.metrics import bleu_score
%load_ext autoreload
%autoreload 2

hypothesis =  [['I', 'ate', 'the', 'apple']]
reference = [['I', 'ate', 'it', 'and', 'that']]
bleu_score(hypothesis, reference, max_n=2, weights=[0.5]*2)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


(0.7788007830714049, 0.40824827551841736, 0.3179440766612941)

In [14]:
matrix = []
for i in range(10):
    src = out_pair[0][0].split('\t')[1].split(' ')
    ref = out_pair[0][1].split('\t')[1].split(' ')
    hyp = out_pair[0][2].split('\t')[2].split(' ')
    if len(src) > 1 and len(ref) > 1 and len(hyp) > 1:
        matrix.append([src, ref, hyp])
arr = np.array(matrix)
arr.shape

(0,)

In [15]:
len(out_pair[:-1])

30116

In [16]:
len(out_pair)

30117

In [17]:
out_pair[-1]

['2020-03-15 16:30:31 | INFO | fairseq_cli.generate | NOTE: hypothesis and token scores are output in base 2',
 '2020-03-15 16:30:31 | INFO | fairseq_cli.generate | Translated 30116 sentences (902896 tokens) in 10490.6s (2.87 sentences/s, 86.07 tokens/s)',
 '2020-03-15 16:30:31 | INFO | fairseq_cli.generate | Generate test with beam=3: BLEU4 = 17.88, 40.7/21.9/13.4/8.5 (BP=1.000, ratio=1.109, syslen=872780, reflen=786915)']

In [123]:
# arr[src, ref, hyp]
# src=en, ref=hyp=fr
matrix = []
for pair in out_pair:
    src = pair[0].split('\t')[1].split(' ')
    ref = pair[1].split('\t')[1].split(' ')
    hyp = pair[2].split('\t')[2].split(' ')
    if len(src) > 1 and len(ref) > 1 and len(hyp) > 1:
        matrix.append([src, ref, hyp])
arr = np.array(matrix)
print(arr.shape)

(29842, 3)


In [19]:
print(len(arr[:,0]),len(arr[:,1]),len(arr[:,2]))

29842 29842 29842


In [20]:
bleu_score(arr[:,2], arr[:,1], max_n=2, weights=[0.5]*2)

(1.0, 0.2988787591457367, 0.2988787591457367)

In [ ]:
bleu_score(arr[:,2], arr[:,1], max_n=2, weights=[0.5]*2)

In [21]:
from models.EditDistance import edit_distance

print(edit_distance(arr[:,2], arr[:,1], normalize=False))
print(edit_distance(arr[:,2], arr[:,1], normalize=True))

103.4395147778299
4.035823287406683


In [43]:
from models import ModelEvaluation as md

a= md.splitDatabySentenceLength(arr, 10, choice='src')
len(a)

17

In [47]:
for count, li in enumerate(a.values()):
    print(f'####### {count*10} words - {len(li)} sentences#######')

####### 0 words - 3322 sentences#######
####### 10 words - 8885 sentences#######
####### 20 words - 8318 sentences#######
####### 30 words - 4907 sentences#######
####### 40 words - 2425 sentences#######
####### 50 words - 1076 sentences#######
####### 60 words - 495 sentences#######
####### 70 words - 225 sentences#######
####### 80 words - 98 sentences#######
####### 90 words - 49 sentences#######
####### 100 words - 17 sentences#######
####### 110 words - 2 sentences#######
####### 120 words - 4 sentences#######
####### 130 words - 5 sentences#######
####### 140 words - 11 sentences#######
####### 150 words - 1 sentences#######
####### 160 words - 2 sentences#######


In [58]:
print(a[0][10])
print(a[0][10][0].count(r'<unk>|<<unk>>'))
print(a[0][10][1].count(r'<unk>|<<unk>>'))

[list(['<unk>', 'relations']) list(['Relations', '<<unk>>'])
 list(['de', 'la', 'sécurité'])]
0
0


In [89]:
from models import ModelEvaluation as md

b= md.splitDatabyNumberOfUnknowns(arr, tick=3, choice='src')
len(b)

12

In [90]:
for count, li in enumerate(b.values()):
    print(f'####### {count*3} unk words - {len(li)} sentences#######')

####### 0 unk words - 13289 sentences#######
####### 3 unk words - 12196 sentences#######
####### 6 unk words - 3335 sentences#######
####### 9 unk words - 763 sentences#######
####### 12 unk words - 180 sentences#######
####### 15 unk words - 52 sentences#######
####### 18 unk words - 1 sentences#######
####### 21 unk words - 16 sentences#######
####### 24 unk words - 1 sentences#######
####### 27 unk words - 7 sentences#######
####### 30 unk words - 1 sentences#######
####### 33 unk words - 1 sentences#######


In [84]:
print(b[3][0][0].count('<unk>'))
b[3][0][0]

39


['Italy',
 '<unk>',
 'by',
 'Mr',
 '<unk>',
 'and',
 '<unk>',
 'on',
 'behalf',
 'of',
 'the',
 'PPE',
 '<unk>',
 'on',
 'the',
 '<unk>',
 'disaster',
 'in',
 'the',
 'Italian',
 'provinces',
 'of',
 '<unk>',
 '<unk>',
 'and',
 '<unk>',
 '<unk>',
 'by',
 'Mr',
 '<unk>',
 'and',
 '<unk>',
 'on',
 'behalf',
 'of',
 'the',
 '<unk>',
 '<unk>',
 'on',
 'the',
 'floods',
 'in',
 '<unk>',
 '<unk>',
 '<unk>',
 'by',
 'Mr',
 '<unk>',
 'on',
 'behalf',
 'of',
 'the',
 '<unk>',
 '<unk>',
 'on',
 'the',
 'tragedy',
 'in',
 'the',
 '<unk>',
 'region',
 'of',
 '<unk>',
 '<unk>',
 'by',
 'Mr',
 '<unk>',
 'and',
 '<unk>',
 'on',
 'behalf',
 'of',
 'the',
 'PSE',
 '<unk>',
 'on',
 'floods',
 'in',
 '<unk>',
 '<unk>',
 'by',
 'Mr',
 '<unk>',
 'and',
 '<unk>',
 'on',
 'behalf',
 'of',
 'the',
 '<unk>',
 '<unk>',
 'on',
 'the',
 'flooding',
 'of',
 '5',
 'May',
 '1998',
 'in',
 '<unk>',
 '<unk>',
 '<unk>',
 'by',
 'Mr',
 '<unk>',
 'and',
 '<unk>',
 'on',
 'behalf',
 'of',
 'the',
 'V',
 '<unk>',
 'on',
 '

In [144]:
data = md.loadFairseqOutput(path)
type(data)

numpy.ndarray

In [124]:
tick=3
c=md.splitDatabyNumberOfUnknowns(data, tick=tick, choice='src')

for key, value in c.items():
    print(f'# of unks: {key*tick} - # of data: {len(value)}')
    arr = np.array(value)
    score = edit_distance(arr[:,2], arr[:,1], normalize=True)
    print(f'Edit distance: {score}\n')

# of unks: 0 - # of sentences: 13289
Edit distance: 3.8952680687995063

# of unks: 3 - # of sentences: 12196
Edit distance: 4.110805181247404

# of unks: 6 - # of sentences: 3335
Edit distance: 4.205797865829334

# of unks: 9 - # of sentences: 763
Edit distance: 4.447225203004894

# of unks: 12 - # of sentences: 180
Edit distance: 4.249541616731467

# of unks: 15 - # of sentences: 52
Edit distance: 4.446389569500742

# of unks: 27 - # of sentences: 1
Edit distance: 5.175

# of unks: 18 - # of sentences: 16
Edit distance: 4.450390840631336

# of unks: 54 - # of sentences: 1
Edit distance: 7.166666666666667

# of unks: 21 - # of sentences: 7
Edit distance: 4.28599286219785

# of unks: 39 - # of sentences: 1
Edit distance: 4.569536423841059

# of unks: 30 - # of sentences: 1
Edit distance: 3.623318385650224



In [142]:
tick=10
d=md.splitDatabySentenceLength(data, tick=tick, choice='src')
for key, value in d.items():
    print(f'sentence length: {key*tick} - # of data: {len(value)}')
    arr = np.array(value)
    score = bleu_score(arr[:,2], arr[:,1])
    print(f'Bleu score: {score}\n')

sentence length: 0 - # of data: 3322
Bleu score: (1.0, 0.14607016742229462, 0.14607016742229462)

sentence length: 10 - # of data: 8885
Bleu score: (1.0, 0.17291192710399628, 0.17291192710399628)

sentence length: 20 - # of data: 8318
Bleu score: (1.0, 0.17774327099323273, 0.17774327099323273)

sentence length: 30 - # of data: 4907
Bleu score: (1.0, 0.18400487303733826, 0.18400487303733826)

sentence length: 40 - # of data: 2425
Bleu score: (1.0, 0.183012455701828, 0.183012455701828)

sentence length: 50 - # of data: 1076
Bleu score: (1.0, 0.18173913657665253, 0.18173913657665253)

sentence length: 60 - # of data: 495
Bleu score: (1.0, 0.18087035417556763, 0.18087035417556763)

sentence length: 70 - # of data: 225
Bleu score: (1.0, 0.18013808131217957, 0.18013808131217957)

sentence length: 80 - # of data: 98
Bleu score: (1.0, 0.18167725205421448, 0.18167725205421448)

sentence length: 90 - # of data: 49
Bleu score: (1.0, 0.17992323637008667, 0.17992323637008667)

sentence length: 100 

In [143]:
tick=10
d=md.splitDatabySentenceLength(data, tick=tick, choice='ref')
for key, value in sorted(d.items(), key=lambda x: x[0]):
    print(f'sentence length: {key*tick} - # of data: {len(value)}')
    arr = np.array(value)
    score = bleu_score(arr[:,2], arr[:,1])
    print(f'Bleu score: {score}\n')

sentence length: 0 - # of data: 3173
Bleu score: (1.0, 0.12927497923374176, 0.12927497923374176)

sentence length: 10 - # of data: 8244
Bleu score: (1.0, 0.1605353057384491, 0.1605353057384491)

sentence length: 20 - # of data: 8103
Bleu score: (1.0, 0.176372230052948, 0.176372230052948)

sentence length: 30 - # of data: 5219
Bleu score: (1.0, 0.18318277597427368, 0.18318277597427368)

sentence length: 40 - # of data: 2706
Bleu score: (1.0, 0.18816982209682465, 0.18816982209682465)

sentence length: 50 - # of data: 1298
Bleu score: (1.0, 0.190720334649086, 0.190720334649086)

sentence length: 60 - # of data: 593
Bleu score: (1.0, 0.18638838827610016, 0.18638838827610016)

sentence length: 70 - # of data: 256
Bleu score: (1.0, 0.19637282192707062, 0.19637282192707062)

sentence length: 80 - # of data: 128
Bleu score: (1.0, 0.18608026206493378, 0.18608026206493378)

sentence length: 90 - # of data: 62
Bleu score: (1.0, 0.19959509372711182, 0.19959509372711182)

sentence length: 100 - # o

In [141]:
for key, value in sorted(d.items(), key=lambda x: x[0]):
    print(key, len(value))

0 3173
1 8244
2 8103
3 5219
4 2706
5 1298
6 593
7 256
8 128
9 62
10 32
11 9
12 5
13 6
14 3
15 2
17 2
22 1


In [135]:
type(sorted(d))

list